### Hello (finance) world!
**Backtesting** einer simplen **Rolling-Window Trading-Strategie auf Basis von Support/Resistance** und der **Relative Strength**. \
Dafür wird der **tägliche bereinigte Schlusskurs** einer beliebigen Aktie in einem definierten Zeitraum von Yahoo! Finance geladen.\
Die benutzten Bibliotheken werden in der folgenden Zelle importiert.

In [5]:
from pandas_datareader import data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.ion()
import statistics as stats
import warnings;
warnings.filterwarnings("ignore")

Eingabe des Start-, End- Datums und Ticker-Symbol der Aktie.\
Datumsformat: "YYYY-MM-DD"

In [2]:
start_date = "2014-01-15"
end_date = "2020-01-01"
stock="GOOG"

Es gibt 2 Signale die einen Trade auslösen: Konsekutives Erreichen von Support/Resistance-Level. \
Als Support/Resistance-Level definiere ich das Maximum/Minimum der letzten 20 Tage $\pm 20$%. \
Falls eines dieser Level an 2 aufeinanderfolgenden Tagen erreicht wird, kaufe/verkaufe ich. \
Desweiteren wird der Simple Moving Average (SMA) als auch der
Relative Strength Indicator \
geplottet errechnet und geplottet.
(über die letzte 20 Tage, für beide).


In [20]:
%matplotlib widget
stock_data = data.DataReader(stock, 'yahoo', start_date, end_date)
stock_data_signal = pd.DataFrame(index=stock_data.index)
stock_data_signal['price'] = stock_data['Adj Close']

def trading_support_resistance(data, rolling_window=20):
    data['sup_tolerance'] = pd.Series(np.zeros(len(data)))
    data['res_tolerance'] = pd.Series(np.zeros(len(data)))
    data['sup_count'] = pd.Series(np.zeros(len(data)))
    data['res_count'] = pd.Series(np.zeros(len(data)))
    data['sup'] = pd.Series(np.zeros(len(data)))
    data['res'] = pd.Series(np.zeros(len(data)))
    data['positions'] = pd.Series(np.zeros(len(data)))
    data['signal'] = pd.Series(np.zeros(len(data)))
    data['sma'] = pd.Series(np.zeros(len(data)))
    in_support=0
    in_resistance=0
    for x in range(2*rolling_window - 1, len(data)):
        data_section = data[x - rolling_window:x + 1]
        support_level=min(data_section['price'])
        resistance_level=max(data_section['price'])
        range_level=resistance_level-support_level
        data['res'][x]=resistance_level
        data['sup'][x]=support_level
        data['sup_tolerance'][x]=support_level + 0.2 * range_level
        data['res_tolerance'][x]=resistance_level - 0.2 * range_level
        data['sma'][x]=stats.mean(data.price[x-20:x])
        if data['price'][x]>=data['res_tolerance'][x] and data['price'][x] <= data['res'][x]:
            in_resistance+=1
            data['res_count'][x]=in_resistance
        elif data['price'][x] <= data['sup_tolerance'][x] and data['price'][x] >= data['sup'][x]:
            in_support += 1
            data['sup_count'][x] = in_support
        else:
            in_support=0
            in_resistance=0
        if in_resistance>2:
            data['signal'][x]=1
        elif in_support>2:
            data['signal'][x]=0
        else:
            data['signal'][x]=data['signal'][x-1]
    data['positions']=data['signal'].diff()

time_period = 20
gain_history = []
loss_history = []
avg_gain_values = []
avg_loss_values = []
rsi_values = []
last_price = 0
for close_price in stock_data_signal.price:
    if last_price == 0:
        last_price = close_price
    gain_history.append(max(0, close_price - last_price))
    loss_history.append(max(0, last_price - close_price))
    last_price = close_price
    if len(gain_history) > time_period:
        del (gain_history[0])
        del (loss_history[0])
    avg_gain = stats.mean(gain_history)
    avg_loss = stats.mean(loss_history)
    avg_gain_values.append(avg_gain)
    avg_loss_values.append(avg_loss)
    rs = 0
    if avg_loss > 0:
        rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    rsi_values.append(rsi)

trading_support_resistance(stock_data_signal)

initial_capital= float(1000.0)
positions = pd.DataFrame(index=stock_data_signal.index).fillna(0.0)
portfolio = pd.DataFrame(index=stock_data_signal.index).fillna(0.0)


positions[stock] = stock_data_signal['signal']
portfolio['positions'] = (positions.multiply(stock_data_signal['price'], axis=0))
portfolio['cash'] = initial_capital - (positions.diff().multiply(stock_data_signal['price'], axis=0)).cumsum()
portfolio['total'] = portfolio['positions'] + portfolio['cash']
portfolio["total"].plot(label="Total portfolio value", ylabel="Value in USD")
stock_data_signal.price.plot(label="Stock price", color="green")
stock_data_signal.sma.plot(label="Simple Moving Average (over 20d)", color="red")
plt.legend()
plt.show()

stock_data_signal=stock_data_signal.assign(RelativeStrengthAvgGainOver20Days=pd.Series(avg_gain_values, index=stock_data_signal.index))
stock_data_signal=stock_data_signal.assign(RelativeStrengthAvgLossOver20Days=pd.Series(avg_loss_values, index=stock_data_signal.index))
stock_data_signal=stock_data_signal.assign(RelativeStrengthIndicatorOver20Days=pd.Series(rsi_values,index=stock_data_signal.index))
close_price = stock_data_signal['price']
rs_gain = stock_data_signal['RelativeStrengthAvgGainOver20Days']
rs_loss = stock_data_signal['RelativeStrengthAvgLossOver20Days']
rsi = stock_data_signal['RelativeStrengthIndicatorOver20Days']


fig = plt.figure()
ax2 = fig.add_subplot(311, ylabel='RS')
rs_gain.plot(ax=ax2, lw=2., legend=True, color="orange")
rs_loss.plot(ax=ax2, lw=2., legend=True, color="cyan")
ax3 = fig.add_subplot(313, ylabel='RSI')
rsi.plot(ax=ax3, color='black', lw=2.,)
plt.legend(loc="best")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …